In [51]:
import noteboard
import pandas as pd
from pathlib import Path

import numpy as np
import networkx as nx

from sklearn.cluster import SpectralClustering

from noteboard.hyperbolicity import CCLHyperbolicity, NaiveHyperbolicity

In [2]:
org_df = pd.read_json("../output/roam_df.json", orient="records", lines=True)

In [3]:
org_df[org_df["level"] == 0]

,org_id,parent_id,name,heading,links,level,path,body,text,modified
7,d17b38ea-44b7-44dd-9378-023ff57f36d1,None,2024-03-13,,[],0,/home/kuba/Projects/org/roam/daily/2024-03-13.org,#+title: 2024-03-13\n,\n#+title: 2024-03-13\n,2024-03-13 14:05:19
8,5d76e47e-5cfe-4f53-a661-e3c32995861a,None,20230227210304-ml_general,,"[96d48b10-7d8e-4e56-b825-1d9930b716ef, bed940f...",0,/home/kuba/Projects/org/roam/20230227210304-ml...,#+title: ml_general\n\ngeneral_nlp\n\nml_artic...,\n#+title: ml_general\n\ngeneral_nlp\n\nml_art...,2024-03-13 14:02:36
9,229c7f84-aae0-4f45-894a-68511ca077f6,None,20240215114344-julia_hyperbolic_embeddings,,[cdf72f48-56ee-4704-baa7-e78d8c831042],0,/home/kuba/Projects/org/roam/20240215114344-ju...,#+title: julia_hyperbolic_embeddings\n\njulia_...,\n#+title: julia_hyperbolic_embeddings\n\njuli...,2024-03-13 13:51:59
12,fc9ca3ad-bd56-428d-8223-71942ae3da4e,None,20231014213642-julia_pluto,,[031da4c3-2782-48d7-865c-36bd2ad18f50],0,/home/kuba/Projects/org/roam/20231014213642-ju...,#+title: julia_pluto\n\njulia\n,\n#+title: julia_pluto\n\njulia\n,2024-03-13 13:51:19
18,38865bff-5060-435b-8b29-66d509bd3423,None,20231012141742-julia_ml,,"[031da4c3-2782-48d7-865c-36bd2ad18f50, 6ac7604...",0,/home/kuba/Projects/org/roam/20231012141742-ju...,#+title: julia_ml\n\n\njulia\n,\n#+title: julia_ml\n\n\njulia\n,2024-03-13 13:51:11
...,...,...,...,...,...,...,...,...,...,...
6426,12038f70-4e59-4766-b7b0-f7a8ade6fba5,None,20230418002749-summarization,,[96d48b10-7d8e-4e56-b825-1d9930b716ef],0,/home/kuba/Projects/org/roam/20230418002749-su...,#+title: summarization\n\ngeneral_nlp\n,\n#+title: summarization\n\ngeneral_nlp\n,2023-09-05 22:28:14
6429,a7e30725-ac43-4c57-a83e-2e6464f63d33,None,20230321163716-work_data_generation,,[6e666966-d8c6-4f57-841e-f2e71fc52322],0,/home/kuba/Projects/org/roam/20230321163716-wo...,#+title: work/data_generation\n\nconversation_...,\n#+title: work/data_generation\n\nconversatio...,2023-09-05 22:28:14
6430,7f0ee414-53fd-44a3-ad1a-57e2ffccd4b4,None,20230410114311-mgr_index_construction,,[43170e6c-9047-4090-8f76-59787cf391d5],0,/home/kuba/Projects/org/roam/20230410114311-mg...,#+title: mgr/index_construction\n\nmgr/represe...,\n#+title: mgr/index_construction\n\nmgr/repre...,2023-09-05 22:28:14
6458,0d15b026-1607-4315-90e4-2892d32475a8,None,20230419131722-wm_mvp,,"[65d4c2ab-8449-497f-9918-11942fa0b76d, a611adc...",0,/home/kuba/Projects/org/roam/20230419131722-wm...,#+title: wm/mvp\n\nwm\n\nwywiad_medyczny_flow\n,\n#+title: wm/mvp\n\nwm\n\nwywiad_medyczny_flow\n,2023-09-05 22:28:14


In [4]:
org_df[org_df["links"].apply(len) > 0]["links"]

8       [96d48b10-7d8e-4e56-b825-1d9930b716ef, bed940f...
9                  [cdf72f48-56ee-4704-baa7-e78d8c831042]
12                 [031da4c3-2782-48d7-865c-36bd2ad18f50]
15                 [6ac76044-a4f7-4fc9-b649-0811ab7c0a5a]
18      [031da4c3-2782-48d7-865c-36bd2ad18f50, 6ac7604...
                              ...                        
6437               [3d62da32-4cbb-40c7-8c9c-d57c121aee16]
6457               [9d540149-3428-42a8-8f45-d3f5c22f784a]
6458    [65d4c2ab-8449-497f-9918-11942fa0b76d, a611adc...
6459               [ab5ccbbf-5893-4d42-8785-6c8e51d0e9e2]
6469               [03023c4a-1650-418d-b67d-e19c7438154a]
Name: links, Length: 1195, dtype: object

In [5]:
def get_org_graph(org_df):
    def get_fname_links(fname_mapping, links):
        return [
            fname_mapping.loc[[link]].iloc[0]["fname"]
            for link in links
            if link in fname_mapping.index
        ]
    
    org_df = org_df.copy()
    org_df["fname"] = org_df["path"].apply(lambda p: Path(p).name)
    fname_mapping = org_df[["fname", "org_id"]].set_index("org_id")
    per_file_links = org_df.groupby("fname")["links"].agg(sum)
    fname_links = per_file_links.apply(
        lambda links: get_fname_links(fname_mapping, links))
    
    # Define a list of node names
    nodes = per_file_links.index
    
    # Define a list of related node names
    edges = [
        (target_id, src_id)
        for (src_id, target_ids) in zip(nodes, fname_links.values)
        for target_id in target_ids
    ]
    
    # Initialize a graph
    org_graph = nx.DiGraph()
    
    # Add nodes and edges to the graph
    org_graph.add_nodes_from(nodes)
    org_graph.add_edges_from(edges)
    return org_graph

In [6]:
org_df[org_df["path"].str.contains("fastrag")]["links"].drop_duplicates()

497               [410d3d4f-673a-4e12-a5b7-bc017a7b7375]
498                                                   []
507    [35740f44-515a-4bf6-bfd1-cd949e827913, 410d3d4...
Name: links, dtype: object

In [7]:
#fname_links[fname_links.apply(lambda links: any("fastrag" in l for l in links))]

In [8]:
full_org_digraph = get_org_graph(org_df)

In [9]:
strongly_connected_components = [c for c in sorted(nx.strongly_connected_components(full_org_digraph), key=len, reverse=True)]

pd.Series([len(c) for c in strongly_connected_components]).describe()

count    1028.000000
mean        1.066148
std         0.573262
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        12.000000
dtype: float64

In [43]:
full_org_graph = full_org_digraph.to_undirected()

full_components = [c for c in sorted(nx.connected_components(full_org_graph), key=len, reverse=True)]

In [49]:
full_components[4]

{'20231128201133-omatko.org', '20231128201140-conferences.org'}

In [11]:
org_graph = full_org_graph.subgraph(full_components[0])
components = full_components = [c for c in sorted(nx.connected_components(org_graph), key=len, reverse=True)]

## Some graph properties

In [12]:
nx.diameter(org_graph)

16

# Exploring central nodes

## Degree centrality

In [14]:
pd.Series(nx.degree_centrality(org_graph)).sort_values(ascending=False).iloc[:25]

20230227210357-llms.org                        0.046344
20230227210304-ml_general.org                  0.035015
20230624202313-mgr.org                         0.029866
20230422190758-inbox.org                       0.028836
20230728175310-math.org                        0.025747
20230805172127-org_ai.org                      0.024717
20230424133658-medsi.org                       0.023687
20230705182132-elixir.org                      0.022657
20230422123611-chatgpt_conversations.org       0.020597
20230227113816-general_nlp.org                 0.020597
20231009191924-julia.org                       0.016478
20230417163110-wm.org                          0.015448
20231018222527-tools.org                       0.015448
20230326130310-llms_langchain.org              0.014418
20230227102218-polish_nlp.org                  0.013388
20230305115720-note_methodology.org            0.013388
20230528181338-project_ideas.org               0.013388
20230430132909-toolz_emacs.org                 0

## Pagerank

In [16]:
pd.Series(nx.pagerank(org_graph)).sort_values(ascending=False).iloc[:25]

20230227210357-llms.org                     0.013413
20230624202313-mgr.org                      0.012373
20230227210304-ml_general.org               0.011360
20230805172127-org_ai.org                   0.009812
20230728175310-math.org                     0.008270
20230705182132-elixir.org                   0.008027
20230422190758-inbox.org                    0.008009
20230422123611-chatgpt_conversations.org    0.007845
20230424133658-medsi.org                    0.007276
20230227113816-general_nlp.org              0.006649
20231018222527-tools.org                    0.005572
20230417163110-wm.org                       0.005520
20231009191924-julia.org                    0.005346
20230430132909-toolz_emacs.org              0.005246
20230227102218-polish_nlp.org               0.004971
20230607161855-rust.org                     0.004656
20230422175934-org_ai.org                   0.004479
20230305115720-note_methodology.org         0.004475
20230911213401-graphs.org                   0.

## Eccentricity

In [18]:
pd.Series(nx.eccentricity(org_graph)).sort_values()

20230623111553-deep_learning_categories.org          9
20231027174420-dimension_estimation.org              9
20231028173653-modeling_mindsets.org                 9
20231015131943-geometrizing_machine_learning.org     9
20230730150956-vpn.org                               9
                                                    ..
20230303221733-mgr_data_size.org                    15
20230520130241-lambda_calculus.org                  15
20231108133003-3d_printing.org                      15
20230226202319-pipelines_from_functions.org         16
20230710121818-optaplanner.org                      16
Length: 972, dtype: int64

## Hyperbolicity

### Reduction to biconnected components

Let $[a,b]$ be any shortest path between $a$ and $b$.

Alternative definition says that $\delta = max_{x,y,z} d([y,z], [x,y] \cup [y,z])$ - the *slim triangles* condition

From slim triangles condition it is easy to see that $\delta = \delta(G) = max_S\ \delta(S)$ where $S$ are the biconnected components.

In [24]:
bi_components = [c for c in sorted(nx.biconnected_components(full_org_graph), key=len, reverse=True)]
org_graph_biggest_bicomponent = org_graph.subgraph(list(bi_components[0]))
org_graph_biggest_bicomponent_dists = nx.floyd_warshall_numpy(org_graph_biggest_bicomponent)

In [42]:
pd.Series([len(c) for c in bi_components]).head()

0    464
1      6
2      4
3      3
4      3
dtype: int64

In [21]:
%%time
delta = CCLHyperbolicity.get_hyperbolicity(org_graph_biggest_bicomponent_dists)

CPU times: user 3min 13s, sys: 380 ms, total: 3min 13s
Wall time: 3min 13s


In [34]:
delta

4.0

## Putting $\delta$ in context

### Diameter bound

The trivial bound is $\delta \leq \frac{diam(G)}{2}$

Note that we only checked the biggest biconnected component because for the other ones $\delta$ is smaller by the lower bound.

So when we normalize by this, $\delta_{norm} = \delta \frac{2}{diam(G)}$

In [31]:
delta_normalized = delta / nx.diameter(org_graph) * 2

In [33]:
delta_normalized

0.5

### Relation to graph size

In *Core-Periphery Models for Graphs Based on their δ-Hyperbolicity* authors propose to relate $\delta$ to graph size.

Their intuition is that if $\delta \leq log_2(V_G)$ then the graph could be considered hyperbolic.

In [38]:
delta / np.log2(len(org_graph.nodes))

0.4030302838010049

## Spectral clustering

Sources: see [A Tutorial on Spectral Clustering](https://people.csail.mit.edu/dsontag/courses/ml14/notes/Luxburg07_tutorial_spectral_clustering.pdf)

In [82]:
SpectralClustering

sklearn.cluster._spectral.SpectralClustering

In [94]:
from typing import Any
from pydantic import BaseModel


class GraphClusterer(BaseModel):
    clusterer: Any
    
    def cluster_graph(self, graph):
        dists = nx.floyd_warshall_numpy(graph)
        return self.clusterer.fit_predict(self.apply_heat_kernel(dists, self.clusterer.gamma))

    def get_clustering_df(self, graph):
        clusters_spectral = self.cluster_graph(graph)
        return pd.DataFrame({"name": graph.nodes(), "cluster_spectral": clusters_spectral}).sort_values("cluster_spectral")
    
    @classmethod
    def apply_heat_kernel(cls, dists, gamma=1.0):
        return np.exp(-gamma * dists / dists.std())

In [95]:
graph_clusterer  = GraphClusterer(clusterer=SpectralClustering(n_clusters=10, affinity="precomputed", gamma=1.0))

In [96]:
spectral_clustering_df = graph_clusterer.get_clustering_df(org_graph)

In [101]:
spectral_clustering_df

,name,cluster_spectral
622,20231009145306-unsupervised_ml.org,0
240,20230502190536-value_networks.org,0
753,20231031115359-ml_evaluation.org,0
752,20231030233942-phase_transitions.org,0
751,20231030233842-breakthroughs.org,0
...,...,...
371,20230612180114-clearml_pipeline_parameters.org,9
370,20230612174359-clearml_pipeline_caching.org,9
363,20230611135315-org_capture_link.org,9
910,20240112115433-sql.org,9
